In [1]:
# imports
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import math
import keras
from scipy.stats  import norm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras        import backend as K
from datetime     import datetime
from keras.optimizers      import RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics       import mean_squared_error
from matplotlib.pylab      import rcParams
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
%matplotlib inline
rcParams['figure.figsize'] = 15, 6

# number of inputs to be fed
look_back    = 288
# number of outputs to be generated
look_forward = 24
# the number of stations
stations     = 66

Using TensorFlow backend.


In [2]:
# The interval between each dataset (original data in 5 minute intervals)
time_grouping = '5min'
# load the dataset
dataframe = pd.read_csv('ignored_assets/paxout_table.csv', engine='python', nrows=288*21)
dataframe['time_bucket'] = pd.to_datetime(dataframe['time_bucket'])
dataframe = dataframe.set_index('time_bucket')
# dataframe['total'] = dataframe.sum(axis=1)
dataframe['day_hour'] = dataframe.index.round(time_grouping)
dataframe = dataframe.groupby('day_hour').sum()
# removes the timestamp at column 67
dataset_orig = dataframe.values[:,:stations]
dataset_orig = dataset_orig.astype('float32')
# scale the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_orig)

# convert an array of values into a dataset matrix, adjusted to make a dateset that is 66 wide
def create_dataset(dataset, look_back=1, look_forward=2):
    dataX, dataY = [], []
    np.array(dataY)
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),:]
        # Makes sure that the array isn't the last 2 or 3 or whatever bits. It has to be the full 24
        if len(dataset[i + look_back:i+look_back+look_forward, 0]) == look_forward:
            dataX.append(a.T)
            dataY.append(dataset[i + look_back:i+look_back+look_forward, :].T)
    return np.array(dataX), np.array(dataY)

# split into train and test sets
train_size = 288*10
test_size = len(dataset) - train_size
test, train = dataset[0:test_size,:], dataset[test_size:len(dataset)-look_forward,:]

# reshape into X=[t, t-1, t-2,..., t-look_back] and Y=[t+1, t+2,... t+look_forward]
trainX, trainY = create_dataset(train, look_back, look_forward)

In [3]:
def top_heavy_loss (y_true, y_pred):
    w = np.arange(1,0, -1./24) ** 2
    w = (w / w.sum())[:,None]
    W = K.variable(value = w)
    return K.dot( K.abs(y_pred-y_true), W)

# Testing Number of Neurons in Dense and LSTM

In [4]:

# creates a model but does not compile the model
def create_model(neurons, LSTM_neurons, layers):
    dropout_rate      = 0.3
    model = Sequential()
    model.add(Dropout(dropout_rate, input_shape=(stations, look_back)))
    lstm = LSTM(neurons, recurrent_dropout=dropout_rate, return_sequences=True)
    model.add(lstm)
    for i in range(layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0), metrics=['accuracy'])
    return model

# creates the model
model = KerasRegressor(build_fn=create_model, epochs=5, batch_size=100, verbose=3)

# The Parameters to Search Through
neuron = [16, 32, 64]
lstm_n = [64, 128, 256]
layers = [4, 5, 6]

# Defines it as a dictionary
param_grid = dict(neurons = neuron,
                  LSTM_neurons = lstm_n,
                  layers = layers)

# Begins the Search For the Optimal Properties
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=20)

grid_result = grid.fit(trainX, trainY)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

K.clear_session()

Epoch 1/5
Epoch 1/5
Epoch 1/5
Epoch 1/5
Epoch 2/5
Epoch 2/5
Epoch 1/5
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 3/5
Epoch 2/5
Epoch 1/5
Epoch 1/5
Epoch 3/5
Epoch 4/5
Epoch 4/5
Epoch 2/5
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 5/5
Epoch 4/5
Epoch 1/5
Epoch 5/5
Epoch 1/5
Epoch 5/5
Epoch 3/5
Epoch 3/5
Epoch 4/5
Epoch 2/5
Epoch 1/5
Epoch 2/5
Epoch 1/5
Epoch 2/5
Epoch 4/5
Epoch 2/5
Epoch 4/5
Epoch 5/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 2/5
Epoch 5/5
Epoch 2/5
Epoch 5/5
Epoch 3/5
Epoch 3/5
Epoch 3/5
Epoch 1/5
Epoch 1/5
Epoch 5/5
Epoch 4/5
Epoch 3/5
Epoch 3/5
Epoch 3/5
Epoch 5/5
Epoch 1/5
Epoch 4/5
Epoch 1/5
Epoch 2/5
Epoch 1/5
Epoch 4/5
Epoch 4/5
Epoch 2/5
Epoch 4/5
Epoch 5/5
Epoch 1/5
Epoch 4/5
Epoch 3/5
Epoch 3/5
Epoch 5/5
Epoch 2/5
Epoch 1/5
Epoch 5/5
Epoch 4/5
Epoch 2/5
Epoch 5/5
Epoch 2/5
Epoch 1/5
Epoch 4/5
Epoch 3/5
Epoch 2/5
Epoch 5/5
Epoch 5/5
Epoch 2/5
Epoch 1/5
Epoch 2/5
Epoch 1/5
Epoch 4/5
Epoch 5/5
Epoch 3/5
Epoch 3/5
Epoch 3/5
Epoch 3/5
Epoch 5/5
Epoch 3/5
Epoch 1/5
Epoch 1/5
Epoch 4/5


In [ ]:
'''
With 6 Layers:

Best: 0.141758 using {'neurons': 32, 'LSTM_neurons': 64}
0.141758 (0.007568) with: {'neurons': 32, 'LSTM_neurons': 64}
0.117460 (0.007247) with: {'neurons': 64, 'LSTM_neurons': 64}
0.097244 (0.005585) with: {'neurons': 128, 'LSTM_neurons': 64}
0.140605 (0.007867) with: {'neurons': 32, 'LSTM_neurons': 128}
0.116654 (0.007119) with: {'neurons': 64, 'LSTM_neurons': 128}
0.097971 (0.006100) with: {'neurons': 128, 'LSTM_neurons': 128}
0.141601 (0.007834) with: {'neurons': 32, 'LSTM_neurons': 256}
0.116020 (0.007053) with: {'neurons': 64, 'LSTM_neurons': 256}
0.098231 (0.006619) with: {'neurons': 128, 'LSTM_neurons': 256}

Best: 0.155671 using {'neurons': 16, 'LSTM_neurons': 256}
0.149573 (0.007568) with: {'neurons': 16, 'LSTM_neurons': 32}
0.142551 (0.007703) with: {'neurons': 32, 'LSTM_neurons': 32}
0.122551 (0.007895) with: {'neurons': 64, 'LSTM_neurons': 32}
0.098012 (0.006323) with: {'neurons': 128, 'LSTM_neurons': 32}
0.152146 (0.006774) with: {'neurons': 16, 'LSTM_neurons': 64}
0.141312 (0.012069) with: {'neurons': 32, 'LSTM_neurons': 64}
0.118877 (0.008444) with: {'neurons': 64, 'LSTM_neurons': 64}
0.097002 (0.006131) with: {'neurons': 128, 'LSTM_neurons': 64}
0.154065 (0.010063) with: {'neurons': 16, 'LSTM_neurons': 128}
0.140651 (0.009266) with: {'neurons': 32, 'LSTM_neurons': 128}
0.119611 (0.007163) with: {'neurons': 64, 'LSTM_neurons': 128}
0.097522 (0.005038) with: {'neurons': 128, 'LSTM_neurons': 128}
0.155671 (0.007716) with: {'neurons': 16, 'LSTM_neurons': 256}
0.144996 (0.006225) with: {'neurons': 32, 'LSTM_neurons': 256}
0.117990 (0.005639) with: {'neurons': 64, 'LSTM_neurons': 256}
0.098801 (0.007384) with: {'neurons': 128, 'LSTM_neurons': 256}

Best: 0.153207 using {'neurons': 16, 'LSTM_neurons': 128}
0.152968 (0.007689) with: {'neurons': 16, 'LSTM_neurons': 64}
0.135548 (0.007788) with: {'neurons': 32, 'LSTM_neurons': 64}
0.121516 (0.007333) with: {'neurons': 64, 'LSTM_neurons': 64}
0.153207 (0.008091) with: {'neurons': 16, 'LSTM_neurons': 128}
0.140024 (0.009425) with: {'neurons': 32, 'LSTM_neurons': 128}
0.118541 (0.007667) with: {'neurons': 64, 'LSTM_neurons': 128}
0.152385 (0.007483) with: {'neurons': 16, 'LSTM_neurons': 256}
0.141717 (0.003894) with: {'neurons': 32, 'LSTM_neurons': 256}
0.122235 (0.007573) with: {'neurons': 64, 'LSTM_neurons': 256}

With Varied Layers:

Best: 0.156554 using {'layers': 6, 'neurons': 16, 'LSTM_neurons': 256}
0.156554 (0.004887) with: {'layers': 6, 'neurons': 16, 'LSTM_neurons': 256}
0.154404 (0.008906) with: {'layers': 6, 'neurons': 16, 'LSTM_neurons': 64}
0.153375 (0.010704) with: {'layers': 5, 'neurons': 16, 'LSTM_neurons': 128}
0.151619 (0.009199) with: {'layers': 6, 'neurons': 16, 'LSTM_neurons': 128}
0.150048 (0.007751) with: {'layers': 5, 'neurons': 16, 'LSTM_neurons': 64}
0.149922 (0.006887) with: {'layers': 4, 'neurons': 16, 'LSTM_neurons': 256}
0.148365 (0.000646) with: {'layers': 5, 'neurons': 16, 'LSTM_neurons': 256}
0.146436 (0.012206) with: {'layers': 4, 'neurons': 16, 'LSTM_neurons': 128}
0.144950 (0.007737) with: {'layers': 4, 'neurons': 16, 'LSTM_neurons': 64}
0.143660 (0.007880) with: {'layers': 6, 'neurons': 32, 'LSTM_neurons': 64}
0.141938 (0.007021) with: {'layers': 6, 'neurons': 32, 'LSTM_neurons': 256}
0.141396 (0.007629) with: {'layers': 4, 'neurons': 32, 'LSTM_neurons': 64}
0.140047 (0.006326) with: {'layers': 6, 'neurons': 32, 'LSTM_neurons': 128}
0.139456 (0.007493) with: {'layers': 5, 'neurons': 32, 'LSTM_neurons': 256}
0.135991 (0.013939) with: {'layers': 5, 'neurons': 32, 'LSTM_neurons': 128}
0.135477 (0.007793) with: {'layers': 5, 'neurons': 32, 'LSTM_neurons': 64}
0.134308 (0.011031) with: {'layers': 4, 'neurons': 32, 'LSTM_neurons': 128}
0.130942 (0.005815) with: {'layers': 4, 'neurons': 32, 'LSTM_neurons': 256}
0.120072 (0.007554) with: {'layers': 6, 'neurons': 64, 'LSTM_neurons': 256}
0.118522 (0.007540) with: {'layers': 6, 'neurons': 64, 'LSTM_neurons': 128}
0.118012 (0.008586) with: {'layers': 6, 'neurons': 64, 'LSTM_neurons': 64}
0.115984 (0.007344) with: {'layers': 5, 'neurons': 64, 'LSTM_neurons': 64}
0.113506 (0.007758) with: {'layers': 5, 'neurons': 64, 'LSTM_neurons': 128}
0.110208 (0.006907) with: {'layers': 4, 'neurons': 64, 'LSTM_neurons': 256}
0.109341 (0.007746) with: {'layers': 5, 'neurons': 64, 'LSTM_neurons': 256}
0.108502 (0.009208) with: {'layers': 4, 'neurons': 64, 'LSTM_neurons': 128}
0.105579 (0.006312) with: {'layers': 4, 'neurons': 64, 'LSTM_neurons': 64}
'''

# Tested Epochs And Batches

In [4]:

# creates a model but does not compile the model
def create_model():
    dropout_rate      = 0.3
    layers            = 6
    neurons           = 64
    model = Sequential()
    model.add(Dropout(dropout_rate, input_shape=(stations, look_back)))
    lstm = LSTM(neurons, recurrent_dropout=dropout_rate, return_sequences=True)
    model.add(lstm)
    for i in range(layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0), metrics=['accuracy'])
    return model

# creates the model
model = KerasRegressor(build_fn=create_model, verbose=3)

# The Parameters to Search Through
epochs = [5, 10, 15]
batch = [60, 80, 100]

# Defines it as a dictionary
param_grid = dict(epochs = epochs,
                  batch_size = batch)

# Begins the Search For the Optimal Properties
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=12)

grid_result = grid.fit(trainX, trainY)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

K.clear_session()

Epoch 1/5
Epoch 1/5
Epoch 1/5
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/15
Epoch 1/15
Epoch 1/5
Epoch 1/15
Epoch 2/5
Epoch 2/5
Epoch 1/5
Epoch 2/5
Epoch 1/5
Epoch 2/10
Epoch 2/10
Epoch 2/10
Epoch 2/15
Epoch 2/15
Epoch 3/5
Epoch 2/15
Epoch 2/5
Epoch 3/5
Epoch 3/5
Epoch 2/5
Epoch 2/5
Epoch 3/10
Epoch 3/10
Epoch 3/10
Epoch 3/15
Epoch 3/15
Epoch 3/15
Epoch 3/5
Epoch 4/5
Epoch 4/5
Epoch 4/5
Epoch 3/5
Epoch 3/5
Epoch 4/10
Epoch 4/10
Epoch 4/10
Epoch 4/15
Epoch 4/15
Epoch 4/15
Epoch 4/5
Epoch 5/5
Epoch 5/5
Epoch 4/5
Epoch 5/5
Epoch 4/5
Epoch 5/10
Epoch 5/10
Epoch 5/10
Epoch 5/15
Epoch 5/15
Epoch 5/5
Epoch 5/15
Epoch 5/5
Epoch 5/5
Epoch 6/10
Epoch 6/10
Epoch 6/10
Epoch 6/15
Epoch 6/15
Epoch 6/15
Epoch 7/10
Epoch 7/10
Epoch 7/10
Epoch 1/10
Epoch 1/10
Epoch 7/15
Epoch 7/15
Epoch 7/15
Epoch 8/10
Epoch 1/10
Epoch 8/10
Epoch 1/15
Epoch 8/10
Epoch 1/15
Epoch 1/15
Epoch 8/15
Epoch 8/15
Epoch 2/10
Epoch 8/15
Epoch 2/10
Epoch 9/10
Epoch 2/10
Epoch 9/10
Epoch 2/15
Epoch 9/10
Epoch 2/15
Epoch 2/15
Epoch 9

In [ ]:
'''

With Parameters:
* 3 Layers
* 128 Neurons
* 0.3 Dropout

Best: 0.067377 using {'epochs': 10, 'batch_size': 40}
0.065068 (0.002889) with: {'epochs': 10, 'batch_size': 10}
0.063950 (0.002562) with: {'epochs': 20, 'batch_size': 10}
0.063110 (0.002909) with: {'epochs': 40, 'batch_size': 10}
0.065298 (0.002661) with: {'epochs': 10, 'batch_size': 20}
0.064239 (0.002350) with: {'epochs': 20, 'batch_size': 20}
0.063582 (0.002740) with: {'epochs': 40, 'batch_size': 20}
0.067377 (0.002870) with: {'epochs': 10, 'batch_size': 40}
0.066048 (0.002805) with: {'epochs': 20, 'batch_size': 40}
0.065648 (0.003975) with: {'epochs': 40, 'batch_size': 40}

Best: 0.071474 using {'epochs': 10, 'batch_size': 80}
0.065794 (0.002702) with: {'epochs': 10, 'batch_size': 20}
0.064984 (0.002984) with: {'epochs': 20, 'batch_size': 20}
0.067603 (0.002976) with: {'epochs': 10, 'batch_size': 40}
0.065411 (0.002310) with: {'epochs': 20, 'batch_size': 40}
0.070512 (0.003740) with: {'epochs': 10, 'batch_size': 60}
0.066848 (0.001883) with: {'epochs': 20, 'batch_size': 60}
0.071474 (0.002628) with: {'epochs': 10, 'batch_size': 80}
0.066839 (0.002004) with: {'epochs': 20, 'batch_size': 80}

Best: 0.091043 using {'epochs': 5, 'batch_size': 100}
0.078441 (0.003310) with: {'epochs': 5, 'batch_size': 60}
0.069070 (0.002873) with: {'epochs': 10, 'batch_size': 60}
0.067836 (0.003454) with: {'epochs': 15, 'batch_size': 60}
0.065659 (0.002621) with: {'epochs': 20, 'batch_size': 60}
0.084301 (0.005425) with: {'epochs': 5, 'batch_size': 80}
0.072219 (0.003395) with: {'epochs': 10, 'batch_size': 80}
0.068752 (0.002674) with: {'epochs': 15, 'batch_size': 80}
0.067021 (0.003355) with: {'epochs': 20, 'batch_size': 80}
0.091043 (0.003273) with: {'epochs': 5, 'batch_size': 100}
0.074570 (0.003157) with: {'epochs': 10, 'batch_size': 100}
0.070304 (0.002780) with: {'epochs': 15, 'batch_size': 100}
0.068518 (0.003850) with: {'epochs': 20, 'batch_size': 100}

With Parameters:
* 6 Layers
* 64 Neurons
* 0.3 Dropout

Best: 0.121967 using {'epochs': 5, 'batch_size': 100}
0.121967 (0.007426) with: {'epochs': 5, 'batch_size': 100}
0.113189 (0.007036) with: {'epochs': 5, 'batch_size': 80}
0.104620 (0.006634) with: {'epochs': 5, 'batch_size': 60}
0.092047 (0.005623) with: {'epochs': 10, 'batch_size': 100}
0.087699 (0.005464) with: {'epochs': 10, 'batch_size': 80}
0.080979 (0.004420) with: {'epochs': 15, 'batch_size': 100}
0.080400 (0.005060) with: {'epochs': 10, 'batch_size': 60}
0.077710 (0.004539) with: {'epochs': 15, 'batch_size': 80}
0.073959 (0.004003) with: {'epochs': 15, 'batch_size': 60}

Best: 0.121967 using {'epochs': 5, 'batch_size': 100}
0.121967 (0.007426) with: {'epochs': 5, 'batch_size': 100}
0.113189 (0.007036) with: {'epochs': 5, 'batch_size': 80}
0.104620 (0.006634) with: {'epochs': 5, 'batch_size': 60}
0.092047 (0.005623) with: {'epochs': 10, 'batch_size': 100}
0.087699 (0.005464) with: {'epochs': 10, 'batch_size': 80}
0.080979 (0.004420) with: {'epochs': 15, 'batch_size': 100}
0.080400 (0.005060) with: {'epochs': 10, 'batch_size': 60}
0.077710 (0.004539) with: {'epochs': 15, 'batch_size': 80}
0.073959 (0.004003) with: {'epochs': 15, 'batch_size': 60}
'''

In [5]:
K.clear_session()